In [1]:
import pandas as pd
import random
from collections import defaultdict, namedtuple

In [2]:
# =========================================================================
# PARÂMETROS GERAIS
# =========================================================================
dias_semana = ["SEG", "TER", "QUA", "QUI", "SEX"]
tempos = [1, 2, 3, 4, 5]
NUM_ITERACOES_HEUR = 1000    # número de iterações da busca heurística inicial
NUM_ITERACOES_REFIN = 1000      # número de iterações do ajuste fino

In [3]:
# =========================================================================
# 1) LEITURA DOS ARQUIVOS
# =========================================================================
df_prof = pd.read_excel("professores.xlsx")
df_demanda_raw = pd.read_excel("demanda.xlsx")  # Demanda vem do arquivo "demanda.xlsx"

In [4]:
# =========================================================================
# 2) CRIAR DICIONÁRIO DE DISPONIBILIDADE
#    disponib[prof][turno][dia][tempo] = True/False
# =========================================================================
disponib = {}
for i, row in df_prof.iterrows():
    servidor = row["SERVIDOR"]
    turno    = row["TURNO"]
    if servidor not in disponib:
        disponib[servidor] = {}
    if turno not in disponib[servidor]:
        disponib[servidor][turno] = {}
    for d in dias_semana:
        if d not in disponib[servidor][turno]:
            disponib[servidor][turno][d] = {}
        for t in tempos:
            col = f"{d}{t}"  # Ex.: SEG1, SEG2, ..., SEX5
            if col in row:
                val = row[col]
                disponib[servidor][turno][d][t] = (val == 1)
            else:
                disponib[servidor][turno][d][t] = False

In [5]:
# =========================================================================
# 3) FILTRAR "APOIO" E MONTAR DEMANDA
# =========================================================================
df_demand = df_demanda_raw[df_demanda_raw["DISC"] != "APOIO"].copy()
Pedido = namedtuple("Pedido", "prof turno nivel serie turma disc ch")
demanda = []
for i, row in df_demand.iterrows():
    demanda.append(Pedido(
        prof  = row["SERVIDOR"],
        turno = row["TURNO"],
        nivel = row["NIVEL"],
        serie = str(row["SERIE/ANO"]),
        turma = str(row["TURMA"]),
        disc  = row["DISC"],
        ch    = int(row["CH"])
    ))

In [6]:
# =========================================================================
# 4) ESTRUTURA DE ALOCAÇÃO
#    horario[turno][nivel][dia][(serie, turma)] = { tempo: (prof, disc) }
# =========================================================================
def cria_horario_vazio(demanda):
    """Gera a estrutura vazia para todos os (turno, nivel, dia, serie, turma) presentes na demanda."""
    horario = {}
    combos = set()
    for ped in demanda:
        combos.add((ped.turno, ped.nivel))
    for (tnr, niv) in combos:
        if tnr not in horario:
            horario[tnr] = {}
        if niv not in horario[tnr]:
            horario[tnr][niv] = {}
        for d in dias_semana:
            horario[tnr][niv][d] = {}
    for ped in demanda:
        for d in dias_semana:
            if (ped.serie, ped.turma) not in horario[ped.turno][ped.nivel][d]:
                horario[ped.turno][ped.nivel][d][(ped.serie, ped.turma)] = {}
    return horario

In [7]:
# =========================================================================
# 5A) FUNÇÕES DE VERIFICAÇÃO DAS RESTRIÇÕES
# =========================================================================
def professor_disponivel(prof, turno, dia, tempo):
    if prof not in disponib:
        return False
    if turno not in disponib[prof]:
        return False
    return disponib[prof][turno][dia][tempo]

def professor_ocupado_esse_tempo(horario, prof, turno, niv, dia, tempo):
    for (st, dicT) in horario[turno][niv][dia].items():
        if tempo in dicT:
            (p, _) = dicT[tempo]
            if p == prof:
                return True
    return False

def turma_ja_tem_aula(horario, turno, niv, dia, serie, turma, tempo):
    return (tempo in horario[turno][niv][dia][(serie, turma)])

def check_buraco(horario, ped, dia):
    dicT = horario[ped.turno][ped.nivel][dia][(ped.serie, ped.turma)]
    if not dicT:
        return False
    tempos_ocup = sorted(dicT.keys())
    if tempos_ocup[0] != 1:
        return True
    maxi = max(tempos_ocup)
    for x in range(1, maxi+1):
        if x not in dicT:
            return True
    return False

def professor_turma_ja_usou_dia(horario, ped, dia):
    """Evita que o mesmo professor seja alocado mais de uma vez por dia na mesma turma."""
    dicTurma = horario[ped.turno][ped.nivel][dia][(ped.serie, ped.turma)]
    for (tm, (p, _)) in dicTurma.items():
        if p == ped.prof:
            return True
    return False

def alocar_1tempo(horario, ped, dia, tempo):
    horario[ped.turno][ped.nivel][dia][(ped.serie, ped.turma)][tempo] = (ped.prof, ped.disc)

def desalocar_1tempo(horario, ped, dia, tempo):
    dicTurma = horario[ped.turno][ped.nivel][dia][(ped.serie, ped.turma)]
    if tempo in dicTurma:
        del dicTurma[tempo]

def pode_alocar(horario, ped, dia, tempo):
    """Verifica todas as restrições antes de alocar."""
    if not professor_disponivel(ped.prof, ped.turno, dia, tempo):
        return False
    if professor_ocupado_esse_tempo(horario, ped.prof, ped.turno, ped.nivel, dia, tempo):
        return False
    if professor_turma_ja_usou_dia(horario, ped, dia):
        return False
    # Testa alocação temporária para checar se gera buraco
    alocar_1tempo(horario, ped, dia, tempo)
    if check_buraco(horario, ped, dia):
        desalocar_1tempo(horario, ped, dia, tempo)
        return False
    desalocar_1tempo(horario, ped, dia, tempo)  # Remove a alocação teste
    return True

In [8]:
# =========================================================================
# 5B) FUNÇÃO PARA CALCULAR CARGA HORÁRIA ALOCADA POR SERVIDOR
# =========================================================================
def calcula_carga_horaria_alocada(horario):
    carga_horaria = defaultdict(int)
    for turno in horario:
        for nivel in horario[turno]:
            for dia in dias_semana:
                for (_, turma), tempos in horario[turno][nivel][dia].items():
                    for tempo, (prof, _) in tempos.items():
                        carga_horaria[prof] += 1
    print("\nCarga Horária Alocada por Servidor:")
    for prof, horas in sorted(carga_horaria.items(), key=lambda x: x[1], reverse=True):
        print(f"{prof}: {horas} aulas")

In [9]:
# =========================================================================
# 6A) BUSCA HEURÍSTICA INICIAL
# =========================================================================
def alocar_demanda(horario, demanda):
    nao_alocados = []
    dem_local = list(demanda)
    random.shuffle(dem_local)
    for ped in dem_local:
        ch_rest = ped.ch
        dias_rand = list(dias_semana)
        random.shuffle(dias_rand)
        for d in dias_rand:
            if ch_rest <= 0:
                break
            tempos_rand = list(tempos)
            random.shuffle(tempos_rand)
            for t in tempos_rand:
                if ch_rest <= 0:
                    break
                if pode_alocar(horario, ped, d, t):
                    alocar_1tempo(horario, ped, d, t)
                    ch_rest -= 1
        if ch_rest > 0:
            nao_alocados.append((ped, ch_rest))
    return nao_alocados

def constroi_solucao(demanda):
    hor_temp = cria_horario_vazio(demanda)
    nao_aloc = alocar_demanda(hor_temp, demanda)
    return hor_temp, nao_aloc


Carga horária alocada após a heurística inicial:

Carga Horária Alocada por Servidor:
JOELMA DA SILVA ARAUJO: 12 aulas
JOYCE PEREIRA FAGUNDES: 12 aulas
TYSSYANNY PEREIRA JARD: 12 aulas
SUELEN CRISTINA DE SOU: 12 aulas
RODRIGO RODRIGUES COLA: 12 aulas
LUANA CAVALLEIRO DE MA: 12 aulas
LAURA CRISTINA PEREIRA: 11 aulas
CLAUDENOR DE SOUZA PIE: 11 aulas
AMARILDO SENA DE FARIA: 10 aulas
JUCINEY DA SILVA FREIT: 10 aulas
AVONEIDE DA SILVA MEND: 9 aulas
THATIANE SILVA DE LIMA: 9 aulas
KAROLINA: 9 aulas
SUELY DOS SANTOS ALMEI: 9 aulas
WELIGTHON JOSE MARTINS: 9 aulas
RAELESON LIMA COELHO: 9 aulas
DANIEL ESTEVES RAID: 9 aulas
CHRISTINA SIMAS CORREA: 9 aulas
EDIVALDO DE SOUZA OLIV: 8 aulas
AYRTON LUCAS LIMA TELE: 8 aulas
JOSE OTONI RAPOSO DIOG: 8 aulas
ELIZABETH DE OLIVEIRA: 8 aulas
GLAURIA GLEICE GAMA DO: 8 aulas
ANDREA CRISTINA NASCIM: 8 aulas
PRISCILA VIANA DE ARAU: 7 aulas
PETRUCIA DE MELO BANDE: 7 aulas
MARIA DO PERPETUO SOCO: 5 aulas
VAGA 01: 5 aulas
RENATA LIMA BORGES FRE: 5 aulas
TATIANE DE

In [10]:
# =========================================================================
# 6B) EXTRAÇÃO DE FEATURES E AVALIAÇÃO
# =========================================================================
def conta_alocacoes(horario):
    used = 0
    for tnr in horario:
        for niv in horario[tnr]:
            for d in dias_semana:
                for (ser, tur), dic in horario[tnr][niv][d].items():
                    used += len(dic)
    return used

def total_slots_ideal(demanda):
    """Cada turma tem 25 slots (5 dias x 5 tempos)."""
    turmas = set((ped.serie, ped.turma) for ped in demanda)
    return len(turmas) * 25

def total_gaps(horario):
    gaps = 0
    for tnr in horario:
        for niv in horario[tnr]:
            for d in dias_semana:
                for (ser, tur), dic in horario[tnr][niv][d].items():
                    if dic:
                        tempos_ocup = sorted(dic.keys())
                        maxi = max(tempos_ocup)
                        for t in range(1, maxi+1):
                            if t not in dic:
                                gaps += 1
    return gaps

def extrair_features(horario, demanda):
    usados = conta_alocacoes(horario)
    slots = total_slots_ideal(demanda)
    gaps = total_gaps(horario)
    nao_aloc = sum(p.ch for p in demanda) - usados
    percent_filled = (usados / slots) * 100 if slots else 0
    return {
        "usados": usados,
        "slots": slots,
        "gaps": gaps,
        "nao_alocados": nao_aloc,
        "percent_filled": percent_filled
    }

def score_solucao(horario, demanda):
    feats = extrair_features(horario, demanda)
    score = feats["usados"] - 2 * feats["gaps"] - 5 * feats["nao_alocados"]
    return score, feats

In [11]:
# =========================================================================
# 6C) ITERAÇÃO E APURAMENTO: BUSCA HEURÍSTICA COM EXTRAÇÃO DE FEATURES
#    Agora, mantemos as N melhores soluções.
# =========================================================================
N_BEST = 10
solucoes = []  # lista para armazenar (horario, nao_alocados, score, features)
for i in range(NUM_ITERACOES_HEUR):
    hor_cand, nao_aloc_cand = constroi_solucao(demanda)
    score, feats = score_solucao(hor_cand, demanda)
    solucoes.append((hor_cand, nao_aloc_cand, score, feats))
# Ordena pelas melhores pontuações (maior score)
solucoes.sort(key=lambda x: x[2], reverse=True)
best_solucoes = solucoes[:N_BEST]

print("Melhores soluções iniciais (heurística):")
for idx, sol in enumerate(best_solucoes, 1):
    print(f"  Solução {idx}: Score = {sol[2]}")

Melhores soluções iniciais (heurística):
  Solução 1: Score = -670
  Solução 2: Score = -676
  Solução 3: Score = -682
  Solução 4: Score = -688
  Solução 5: Score = -688
  Solução 6: Score = -688
  Solução 7: Score = -694
  Solução 8: Score = -700
  Solução 9: Score = -700
  Solução 10: Score = -700


In [12]:
# =========================================================================
# 6D) AJUSTE FINO (REFINEMENT) SOBRE AS N MELHORES SOLUÇÕES
#    Para cada solução, tenta forçar a alocação dos tempos remanescentes.
# =========================================================================
def ajuste_fino(horario, nao_alocados):
    novos_nao = []
    for ped, ch_rest in nao_alocados:
        # Tenta alocar para cada pedido, utilizando somente dias onde o professor ainda não foi alocado na turma
        for d in dias_semana:
            if ch_rest <= 0:
                break
            # Só tenta se o professor não foi alocado neste dia para essa turma
            if professor_turma_ja_usou_dia(horario, ped, d):
                continue
            for t in tempos:
                if ch_rest <= 0:
                    break
                if pode_alocar(horario, ped, d, t):
                    alocar_1tempo(horario, ped, d, t)
                    ch_rest -= 1
        if ch_rest > 0:
            novos_nao.append((ped, ch_rest))
    return horario, novos_nao

refined_solucoes = []
for hor, nao_aloc, score_init, feats_init in best_solucoes:
    # Refinamento: iterar NUM_ITERACOES_REFIN vezes refinando essa solução
    for _ in range(NUM_ITERACOES_REFIN):
        hor, novos_nao = ajuste_fino(hor, nao_aloc)
        nao_aloc = novos_nao  # atualiza a lista de pedidos não alocados
    score_ref, feats_ref = score_solucao(hor, demanda)
    refined_solucoes.append((hor, nao_aloc, score_ref, feats_ref))

# Seleciona a melhor solução refinada
refined_solucoes.sort(key=lambda x: x[2], reverse=True)
melhor_refinado, melhor_nao_alocados, melhor_score_ref, feats_refin = refined_solucoes[0]

print("Resultado após ajuste fino (refinamento das N melhores soluções):")
print(f"Melhor Score Refinado: {melhor_score_ref}")
usados_final = conta_alocacoes(melhor_refinado)
total_final = total_slots_ideal(demanda)
percent_filled = (usados_final / total_final) * 100 if total_final else 0
print(f"Slots usados: {usados_final}")
print(f"Slots vazios: {total_final - usados_final}")
print(f"Percentual de preenchimento: {percent_filled:.2f}%")
print("----------------------------------------------")

Resultado após ajuste fino (refinamento das N melhores soluções):
Melhor Score Refinado: -622
Slots usados: 313
Slots vazios: 187
Percentual de preenchimento: 62.60%
----------------------------------------------


In [16]:
# =========================================================================
# CÓDIGO ADICIONAL: RESUMO DA ALOCAÇÃO DE CH POR SERVIDOR
# =========================================================================
# Calcula o total de CH demandado por servidor (soma dos pedidos)
total_CH_por_servidor = {}
for ped in demanda:
    total_CH_por_servidor[ped.prof] = total_CH_por_servidor.get(ped.prof, 0) + ped.ch

# Calcula a CH alocada em cada slot do melhor horário refinado
CH_alocada_por_servidor = {}
for tnr in melhor_refinado:
    for niv in melhor_refinado[tnr]:
        for d in dias_semana:
            for (serie, turma), dic in melhor_refinado[tnr][niv][d].items():
                for t in dic:
                    prof, _ = dic[t]
                    CH_alocada_por_servidor[prof] = CH_alocada_por_servidor.get(prof, 0) + 1

# Monta o resumo com a CH demandada, alocada e o saldo (não alocado)
resumo_CH = []
for prof in total_CH_por_servidor:
    total_demandado = total_CH_por_servidor[prof]
    alocada = CH_alocada_por_servidor.get(prof, 0)
    nao_alocada = total_demandado - alocada
    resumo_CH.append({
        "Servidor": prof,
        "Total CH Demandado": total_demandado,
        "CH Alocada": alocada,
        "CH Não Alocada": nao_alocada
    })

df_resumo_CH = pd.DataFrame(resumo_CH)

# Adiciona uma linha final de total utilizando pd.concat
total_linha = {
    "Servidor": "TOTAL",
    "Total CH Demandado": df_resumo_CH["Total CH Demandado"].sum(),
    "CH Alocada": df_resumo_CH["CH Alocada"].sum(),
    "CH Não Alocada": df_resumo_CH["CH Não Alocada"].sum()
}
df_total = pd.DataFrame([total_linha])
df_resumo_CH = pd.concat([df_resumo_CH, df_total], ignore_index=True)

print("Resumo da alocação de CH por servidor:")
print(df_resumo_CH)



Resumo da alocação de CH por servidor:
                  Servidor  Total CH Demandado  CH Alocada  CH Não Alocada
0   AMARILDO SENA DE FARIA                  15          12               3
1   ANDREA CRISTINA NASCIM                  18          13               5
2   AVONEIDE DA SILVA MEND                  19          13               6
3   AYRTON LUCAS LIMA TELE                  16           7               9
4   CHRISTINA SIMAS CORREA                  16          11               5
5   CLAUDENOR DE SOUZA PIE                  16          10               6
6      DANIEL ESTEVES RAID                  15           8               7
7   EDIVALDO DE SOUZA OLIV                  16          12               4
8    ELIZABETH DE OLIVEIRA                  16           9               7
9                    ERIKA                   7           2               5
10  GLAURIA GLEICE GAMA DO                  16          11               5
11  JOELMA DA SILVA ARAUJO                  16          11   

In [14]:
# =========================================================================
# 6E) SALVAR FEATURES EXTRAÍDAS (para uso em ML futuro)
# =========================================================================
df_features = pd.DataFrame([{"Score": sol[2], **sol[3]} for sol in refined_solucoes])
df_features.to_excel("FEATURES_SOLUCOES.xlsx", index=False)

In [17]:
# =========================================================================
# 7) EXPORTAÇÃO FINAL DOS RESULTADOS
#    (A) Quadro de Horários por Turma (abas por (TURNO, NIVEL))
#    (B) Quadro de Horários por Professor (aba por professor)
#    (C) Lista Detalhada dos Slots Vazios
# =========================================================================
def gera_df_por_turma(horario):
    colunas_por_tn = defaultdict(set)
    for tnr in horario:
        for niv in horario[tnr]:
            for d in dias_semana:
                for (ser, tur), dic in horario[tnr][niv][d].items():
                    colunas_por_tn[(tnr, niv)].add(f"{ser}-{tur}")
    dfs_por_aba = {}
    for tnr in horario:
        for niv in horario[tnr]:
            turmas_cols = sorted(list(colunas_por_tn[(tnr, niv)]))
            linhas = []
            for d in dias_semana:
                for tm in tempos:
                    row_dict = {"TURNO": tnr, "NIVEL": niv, "DIA": d, "TEMPO": tm}
                    for col in turmas_cols:
                        row_dict[col] = ""
                    for (ser, tur), dic in horario[tnr][niv][d].items():
                        if tm in dic:
                            (p, ds) = dic[tm]
                            colname = f"{ser}-{tur}"
                            row_dict[colname] = f"{p}+{ds}"
                    linhas.append(row_dict)
            df_aba = pd.DataFrame(linhas)
            dfs_por_aba[f"{tnr}_{niv}"] = df_aba
    return dfs_por_aba

def gera_df_por_professor(horario):
    prof_data = defaultdict(lambda: defaultdict(list))
    prof_cols = defaultdict(set)
    for tnr in horario:
        for niv in horario[tnr]:
            for d in dias_semana:
                for (ser, tur), dic in horario[tnr][niv][d].items():
                    for tm, (p, ds) in dic.items():
                        col = f"{ser}-{tur}({ds})"
                        prof_cols[p].add(col)
                        prof_data[p][(d, tm)].append((tnr, niv, ser, tur, ds))
    dfs_por_prof = {}
    for p in prof_data:
        cols = sorted(list(prof_cols[p]))
        linhas = []
        for d in dias_semana:
            for tm in tempos:
                row_dict = {"PROFESSOR": p, "DIA": d, "TEMPO": tm}
                for c in cols:
                    row_dict[c] = ""
                if (d, tm) in prof_data[p]:
                    for (tnr, niv, ser, tur, ds) in prof_data[p][(d, tm)]:
                        cname = f"{ser}-{tur}({ds})"
                        row_dict[cname] = f"{tnr}-{niv}"
                linhas.append(row_dict)
        df_p = pd.DataFrame(linhas)
        dfs_por_prof[p] = df_p
    return dfs_por_prof

dfs_turma = gera_df_por_turma(melhor_refinado)
dfs_prof = gera_df_por_professor(melhor_refinado)

with pd.ExcelWriter("QUADRO_HORÁRIOS_TURMAS.xlsx") as writer:
    for aba, df_ in dfs_turma.items():
        df_.to_excel(writer, sheet_name=aba[:31], index=False)

with pd.ExcelWriter("QUADRO_HORÁRIOS_PROFESSORES.xlsx") as writer:
    for p, df_ in dfs_prof.items():
        sheetname = p[:31]
        df_.to_excel(writer, sheet_name=sheetname, index=False)

vazios_detalhe = []
for tnr in melhor_refinado:
    for niv in melhor_refinado[tnr]:
        for d in dias_semana:
            for (ser, tur), dic in melhor_refinado[tnr][niv][d].items():
                for tm in tempos:
                    if tm not in dic:
                        vazios_detalhe.append({
                            "TURNO": tnr,
                            "NIVEL": niv,
                            "DIA": d,
                            "SERIE": ser,
                            "TURMA": tur,
                            "TEMPO": tm
                        })

df_vaz = pd.DataFrame(vazios_detalhe)
df_vaz.to_excel("NAO_ALOCADOS_TEMPOS_VAZIOS.xlsx", index=False)

print("Arquivos gerados:")
print(" - QUADRO_HORÁRIOS_TURMAS.xlsx (abas por Turno_Nível)")
print(" - QUADRO_HORÁRIOS_PROFESSORES.xlsx (abas por professor)")
print(" - NAO_ALOCADOS_TEMPOS_VAZIOS.xlsx (lista detalhada dos slots vazios)")
print(" - FEATURES_SOLUCOES.xlsx (features de cada iteração)")

Arquivos gerados:
 - QUADRO_HORÁRIOS_TURMAS.xlsx (abas por Turno_Nível)
 - QUADRO_HORÁRIOS_PROFESSORES.xlsx (abas por professor)
 - NAO_ALOCADOS_TEMPOS_VAZIOS.xlsx (lista detalhada dos slots vazios)
 - FEATURES_SOLUCOES.xlsx (features de cada iteração)
